In [1]:
# imports

import os
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import pickle
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
from items import Item
from testing import Tester
from agents.pricer_agent import price
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# CONSTANTS

QUESTION = "How much does this cost to the nearest dollar?\n\n"
DB = "products_vectorstore"

In [3]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [4]:
openai = OpenAI()

In [5]:
# Load in the test pickle file:

with open('test.pkl', 'rb') as file:
    test = pickle.load(file)

In [6]:
def make_context(similars, prices):
    message = "To provide some context, here are some other items that might be similar to the item you need to estimate.\n\n"
    for similar, price in zip(similars, prices):
        message += f"Potentially related product:\n{similar}\nPrice is ${price:.2f}\n\n"
    return message

In [ ]:
def messages_for(item, similars, prices):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = make_context(similars, prices)
    user_prompt += "And now the question for you:\n\n"
    user_prompt += item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "Price is $"}
    ]

In [ ]:
client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection('products')

In [ ]:
def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
def vector(item):
    return model.encode([description(item)])

In [ ]:
def find_similars(item):
    results = collection.query(query_embeddings=vector(item).astype(float).tolist(), n_results=5)
    documents = results['documents'][0][:]
    prices = [m['price'] for m in results['metadatas'][0][:]]
    return documents, prices

In [ ]:
def get_price(s):
    s = s.replace('$','').replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    return float(match.group()) if match else 0

In [ ]:
# The function for gpt-4o-mini

def gpt_4o_mini_rag(item):
    documents, prices = find_similars(item)
    response = openai.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages_for(item, documents, prices),
        seed=42,
        max_tokens=5
    )
    reply = response.choices[0].message.content
    return get_price(reply)

In [ ]:
gpt_4o_mini_rag(test[1000])

In [ ]:
test[1000]

In [ ]:
def proprietary(item):
    text = item.prompt.split("to the nearest dollar?\n\n")[1].split("\n\nPrice is $")[0]
    return price(text)

In [ ]:
print(proprietary(test[1]))
print(gpt_4o_mini_rag(test[1]))

In [ ]:
truths = []
proprietaries = []
rags = []
for i in tqdm(range(1000,1250)):
    item = test[i]
    truths.append(item.price)
    proprietaries.append(proprietary(item))
    rags.append(gpt_4o_mini_rag(item))

In [ ]:
mins = [min(p,r) for p,r in zip(proprietaries, rags)]
maxes = [max(p,r) for p,r in zip(proprietaries, rags)]

X = pd.DataFrame({
    'Proprietary': proprietaries,
    'RAG': rags,
    'Min': mins,
    'Max': maxes,
})

# Convert y to a Series
y = pd.Series(truths)

In [ ]:
# Train a Linear Regression
np.random.seed(42)

lr = LinearRegression()
lr.fit(X, y)

feature_columns = ["Proprietary", "RAG", "Min", "Max"]

for feature, coef in zip(feature_columns, lr.coef_):
    print(f"{feature}: {coef:.2f}")
print(f"Intercept={lr.intercept_:.2f}")

In [ ]:
def ensemble(item):
    prop = proprietary(item)
    rag = gpt_4o_mini_rag(item)
    Xt = pd.DataFrame({
    'Proprietary': [prop],
    'RAG': [rag],
    'Min': [min(prop,rag)],
    'Max': [max(prop,rag)],
    })
    yt = lr.predict(Xt)
    return yt[0]

In [ ]:
ensemble(test[0])

In [ ]:
test[0].price

In [ ]:
Tester.test(ensemble, test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
prices = [metadata['price'] for metadata in result['metadatas']]

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=8)
rf_model.fit(vectors, prices)

In [ ]:
def new_rf(item):
    text = item.prompt.split("to the nearest dollar?\n\n")[1].split("\n\nPrice is $")[0]
    vector = model.encode([text])
    return max(0, rf_model.predict(vector)[0])

In [ ]:
new_rf(test[0])

In [ ]:
Tester.test(new_rf, test)

In [ ]:
forests = []
for i in tqdm(range(1000,1250)):
    item = test[i]
    forests.append(new_rf(item))

In [ ]:
truths2 = []
proprietaries2 = []
rags2 = []
forests2 = []
for i in tqdm(range(1000,2000)):
    item = test[i]
    truths2.append(item.price)
    proprietaries2.append(proprietary(item))
    rags2.append(gpt_4o_mini_rag(item))
    forests2.append(new_rf(item))

In [ ]:
mins2 = [min(p,r,f) for p,r,f in zip(proprietaries2, rags2, forests2)]
maxes2 = [max(p,r,f) for p,r,f in zip(proprietaries2, rags2, forests2)]



X2 = pd.DataFrame({
    'Proprietary': proprietaries2,
    'RAG': rags2,
    'Forest': forests2,
    'Min': mins2,
    'Max': maxes2,
})

# Convert y to a Series
y2 = pd.Series(truths2)

In [ ]:
# Train a Linear Regression
np.random.seed(42)

lr2 = LinearRegression()
lr2.fit(X2, y2)

feature_columns = X2.columns.tolist()

for feature, coef in zip(feature_columns, lr2.coef_):
    print(f"{feature}: {coef:.2f}")
print(f"Intercept={lr.intercept_:.2f}")

In [ ]:
def ensemble2(item):
    prop = proprietary(item)
    rag = gpt_4o_mini_rag(item)
    r_f = new_rf(item)
    Xt2 = pd.DataFrame({
    'Proprietary': [prop],
    'RAG': [rag],
    'Forest': [r_f],
    'Min': [min(prop,rag, r_f)],
    'Max': [max(prop,rag, r_f)],
    })
    yt2 = lr.predict(Xt2)
    return yt2[0]

In [ ]:
Tester.test(ensemble2, test)